In [ ]:
import os ##to store path of image
import tensorflow as tf  
from tensorflow.keras.models import load_model
from keras_facenet import FaceNet
import numpy as np
import pickle ##type of storage method
import cv2
import PIL ##convert image to array
from PIL import Image

In [2]:
HaarCascade = cv2.CascadeClassifier('haarcascade-frontalface-default.xml')
facenet = FaceNet()

In [3]:
folder='Pictures/'
database_file = 'Database.pickle'

In [4]:
def face_to_embeddings(Img): ##1D representation of vectors with 128 values
    face = HaarCascade.detectMultiScale(Img,1.1,4)
    
    if len(face)>0:
        x1, y1, width, height = face[0]         
    else:
        x1, y1, width, height = 1, 1, 5, 5
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    Img = cv2.cvtColor(Img, cv2.COLOR_BGR2RGB)
    Img = PIL.Image.fromarray(Img)                 
    Img_array = np.asarray(Img)
    
    face = Img_array[y1:y2, x1:x2]                        
    
    face = PIL.Image.fromarray(face)                       
    face = face.resize((160,160))
    face = np.asarray(face)
    
    face = np.expand_dims(face, axis=0)
    embedd = facenet.embeddings(face)
    embedd = embedd / np.linalg.norm(embedd, ord=2)
    return embedd, x1, y1, x2, y2

In [5]:
if os.path.exists(database_file):
    with open(database_file, 'rb') as f:
        database = pickle.load(f)    
else:
    database = {}

In [6]:
for filename in os.listdir(folder):
    if filename.split('.')[0] not in database.keys():
        path = folder + filename
        gbr1 = cv2.imread(path)
        signature, _, _, _, _ = face_to_embeddings(gbr1)
        database[os.path.splitext(filename)[0]] = signature
        print('Embedding of ' + filename.split('.')[0] + " stored into the database")

with open(database_file, 'wb') as f:
    pickle.dump(database, f)

In [7]:
database.keys()

dict_keys(['Beyonce', 'Chandrahas', 'chris', 'prabhas'])

In [8]:
def verify_face():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        _, Img = cap.read()
        embedd,x1,y1,x2,y2 = face_to_embeddings(Img)
        min_dist=0
        identity=' '
        for key, value in database.items() :
            dist = np.dot(value,embedd.T)/np.linalg.norm(value,ord=2) * np.linalg.norm(embedd,ord=2)
            if dist > min_dist:
                min_dist = dist
                identity = key
        cv2.rectangle(Img,(x1,y1),(x2,y2), (0,255,0), 2)
        Img = cv2.flip(Img,1)
        if min_dist < 0.7:
            cv2.putText(Img,'unknown', (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(Img,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(Img,"            " +str(min_dist), (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('res',Img)
        k = cv2.waitKey(1) & 0xFF 
        if k == 27:
            break
    cv2.destroyAllWindows()
    cap.release()

In [9]:
verify_face()

1/1 [==============================] - 0s 40ms/step


In [31]:
# def record(cap):
#     ret, Img = cap.read()
#     face = HaarCascade.detectMultiScale(Img,1.1,4)
#     if len(face)>0:
#         x1, y1, width, height = face[0]         
#     else:
#         x1, y1, width, height = 1, 1, 10, 10
        
#     x1, y1 = abs(x1), abs(y1)
#     x2, y2 = x1 + width, y1 + height
#     return x1,y1,x2,y2,face,Img

In [10]:
def register_face():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        _, Img = cap.read()
        face = HaarCascade.detectMultiScale(Img,1.1,4)
        if len(face)>0:
            x1, y1, width, height = face[0]         
        else:
            x1, y1, width, height = 1, 1, 5, 5
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        cv2.rectangle(Img,(x1,y1),(x2,y2), (0,255,0), 2)
        Img = cv2.flip (Img,1)
        cv2.imshow('res',Img)
        k = cv2.waitKey(1) & 0xFF 
        if k == 27:    
            ret, Img = cap.read()
            face = HaarCascade.detectMultiScale(Img,1.1,4)
            if len(face)>0:
                x1, y1, width, height = face[0]         
            else:
                x1, y1, width, height = 1, 1, 5, 5
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            break
    cv2.destroyAllWindows()
    cap.release() 
    name = input(" Enter name ")
    ImG = cv2.imwrite('Pictures/' + name + '.jpg',Img)
    Img = cv2.cvtColor(Img, cv2.COLOR_BGR2RGB)
    Img = PIL.Image.fromarray(Img)                 
    Img_array = np.asarray(Img)
    face = Img_array[y1:y2, x1:x2]                        
    face = PIL.Image.fromarray(face)                       
    face = face.resize((160,160))
    face = np.asarray(face)
    face = np.expand_dims(face, axis=0)
    embedd = facenet.embeddings(face)
    embedd = embedd / np.linalg.norm(embedd, ord=2)
    database[name] = embedd
    with open(database_file, 'wb') as f:
        pickle.dump(database, f)

In [11]:
register_face()

1/1 [==============================] - 0s 40ms/step


In [12]:
def delete_face():# write a statement to delete the image from the computer also
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        _, Img = cap.read()
        face = HaarCascade.detectMultiScale(Img,1.1,4)
        if len(face)>0:
            x1, y1, width, height = face[0]         
        else:
            x1, y1, width, height = 1, 1, 5, 5
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        cv2.rectangle(Img,(x1,y1),(x2,y2), (0,255,0), 2)
        Img = cv2.flip (Img,1)
        cv2.imshow('res',Img)
        k = cv2.waitKey(1) & 0xFF 
        if k == 27:    
            ret, Img = cap.read()
            face = HaarCascade.detectMultiScale(Img,1.1,4)
            if len(face)>0:
                x1, y1, width, height = face[0]         
            else:
                x1, y1, width, height = 1, 1, 5, 5
        
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            break
    cv2.destroyAllWindows()
    cap.release() 
    Img = cv2.cvtColor(Img, cv2.COLOR_BGR2RGB)
    Img = PIL.Image.fromarray(Img)                 
    Img_array = np.asarray(Img)
    face = Img_array[y1:y2, x1:x2]                        
    face = PIL.Image.fromarray(face)                       
    face = face.resize((160,160))
    face = np.asarray(face)
    face = np.expand_dims(face, axis=0)
    embedd = facenet.embeddings(face)
    embedd = embedd / np.linalg.norm(embedd, ord=2)
    min_dist=0
    identity=' '
    for key, value in database.items() :
        dist = np.dot(value,embedd.T)
        if dist > min_dist:
            min_dist = dist
            identity = key
    if min_dist < 0.7:
        print(" Identity cannot be found")
    else:
        del database[identity]
        print(identity + ' deleted ')
        with open(database_file, 'wb') as f:
            pickle.dump(database, f)

In [13]:
delete_face()

1/1 [==============================] - 0s 57ms/step
Chandrahas 2 deleted 


In [14]:
database.keys()

dict_keys(['B', 'Chris', 'chandrahas', '2.0'])

In [10]:
def view_visitors():
    i=1
    for key in database.keys() :
        print(str(i) +'. '+ key)
        i+=1

In [11]:
view_visitors()

1. B
2. Chris
3. chandrahas
4. 2.0
5. Chandrahas 2
